In [1]:
import os
import pandas as pd
import copy
from IPython.display import HTML
import sqlite3 as db

In [2]:
con = db.connect('C:/Users/cliff/workspace/rushHour/Data Model and SQL/Schema Definition and Bulk Load/rush_hour.db')
c = con.cursor()

In [3]:
%run display_utilities.py
%run compute_edges_connected_components.py

We start with a combinatorial class and derive the following:
     * state transitions
     * connected components

And we update states according to these derivations.

The algorithm:
    
    Start with combinatorial class $C$. 
    
    Compute set $T$ of all topological classes $T_i$ in $C$ along with number of states in $T_i; T =[ [T_1,N_1], [T_2,N_2], ...,[T_n,N_n] ]$
    
    Compute subsets of $T$ such that the number of states within each subset is constrained by a hardcoded constant
    
    For each such subset $S$ of $T$, derive the connected components within $S$
    
    Record those connected components in db

In [4]:
comb_class_id = 216
c.execute("select topo_class_hash,count(*) from states where comb_class_id = %d group by topo_class_hash" %(comb_class_id))
topo_classes = c.fetchall()
topo_class_subsets = topo_classes_subsets_defined_by_state_count(topo_classes)

for subset in topo_class_subsets:
    sql_in_clause = '(' + ','.join([str(x) for x in subset]) + ')'

    sql = """select game_number,game_hash_top,game_hash_bottom,red_car_end_a,topo_class_hash 
             from states 
             where topo_class_hash in """ + sql_in_clause

    c.execute(sql)

    all_states = c.fetchall()
    all_states_dict = { (x[1],x[2],x[3]) : x[0] for x in all_states }
    topo_hash_dict = { (x[1],x[2],x[3]) : x[4] for x in all_states }
    connected_components = compute_connected_components(all_states_dict)
    
    for connected_component in connected_components:
        [states,edges] = connected_component
        game_numbers = list( set( [x[0] for x in edges] + [x[1] for x in edges]))
        game_numbers_in_clause = '(' + ','.join([str(x) for x in game_numbers]) + ')'
        topo_class_hash = topo_hash_dict[states[1]]

        component_sql = "insert into connected_component(num_states,topo_class_hash) values(%d,%d)"%(len(game_numbers), topo_class_hash)
        c.execute(component_sql)
        component_id = c.lastrowid

        update_states_sql = "update states set connected_component_id = %d where comb_class_id = %d and game_number in " %(component_id, comb_class_id)
        update_states_sql = update_states_sql + game_numbers_in_clause
        c.execute(update_states_sql)   

        transitions_for_insert = [ [comb_class_id] + x for x in edges] 
        insert_transitions_sql = "insert into state_transition(comb_class_id, pre_transition_game_number, post_transition_game_number) values(?,?,?)"
        c.executemany(insert_transitions_sql,(transitions_for_insert))
        con.commit()

IndexError: list index out of range

In [23]:
# !!!! TODO - convert this step into a loop through each connected component 
for connected_component in connected_components:
    [states,edges] = connected_component
    game_numbers = list( set( [x[0] for x in edges] + [x[1] for x in edges]))
    game_numbers_in_clause = '(' + ','.join([str(x) for x in game_numbers]) + ')'
    topo_class_hash = topo_hash_dict[states[1]]

    component_sql = "insert into connected_component(num_states,topo_class_hash) values(%d,%d)"%(len(game_numbers), topo_class_hash)
    c.execute(component_sql)
    component_id = c.lastrowid

    update_states_sql = "update states set connected_component_id = %d where comb_class_id = %d and game_number in " %(component_id, comb_class_id)
    update_states_sql = update_states_sql + game_numbers_in_clause
    c.execute(update_states_sql)   

    transitions_for_insert = [ [comb_class_id] + x for x in edges] 
    insert_transitions_sql = "insert into state_transition(comb_class_id, pre_transition_game_number, post_transition_game_number) values(?,?,?)"
    c.executemany(insert_transitions_sql,(transitions_for_insert))
    con.commit()

In [24]:
len(topo_class_subsets)

1